<a href="https://colab.research.google.com/github/toraaglobal/tutorial/blob/text_mining/twitter_mining_for_sentiment_classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Twitter Mining
### Retriving live tweets from twitter using hash tag or keywords and save it in a file.
***

* Get twitter secrete and access key for authorization
Go to twitter developer accout to request for access using this link https://developer.twitter.com/en/products/twitter-api


### Install packages

In [1]:
%%bash
pip install tweepy
pip install wordcloud



### Import packages

In [2]:
import tweepy
from tweepy import OAuthHandler
import json
from tweepy import Stream
from tweepy.streaming import StreamListener
import sys
import json
from nltk.tokenize import word_tokenize
from nltk.tokenize import TweetTokenizer
import re
from os import path
import os
import matplotlib.pyplot as plt
from wordcloud import WordCloud, STOPWORDS
from google.colab import drive

%matplotlib inline

### Mount gdrive to access twitter authentication token and secret

In [5]:
drive.mount('/content/drive', force_remount=True)
os.chdir('./drive/My Drive/tool') # change directory

Mounted at /content/drive


In [7]:

def get_twitter_auth():
	auth = []
	f = open("./auth.txt", "r")
	for line in f:
		auth.append(line.strip())
	f.close()
	return auth


In [8]:
authlist = get_twitter_auth()

consumer_key = authlist[0]
consumer_secret = authlist[1]
access_token = authlist[2]
access_secret = authlist[3]

### Authentication

In [11]:
auth = OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_token, access_secret)
api = tweepy.API(auth)

### Custom class for tweets retrival

In [9]:

class Listener(StreamListener):
    print("In Listener...") 
    tweet_number=0
    def __init__(self, max_tweets, hfilename, rawfile):
        self.max_tweets=max_tweets
        print(self.max_tweets)

    #on_data() is a function of StreamListener as is on_error and on_status    
    def on_data(self, data):
        self.tweet_number+=1 
        print("In on_data", self.tweet_number)
        try:
            print("In on_data in try")
            with open(hfilename, 'a') as f:
                with open(rawfile, 'a') as g:
                    tweet=json.loads(data)
                    tweet_text=tweet["text"]
                    print(tweet_text,"\n")
                    f.write(tweet_text) # the text from the tweet
                    json.dump(tweet, g)  #write the raw tweet
        except BaseException:
            print("NOPE")
            pass
        if self.tweet_number>=self.max_tweets:
            #sys.exit('Limit of '+str(self.max_tweets)+' tweets reached.')
            print("Got ", str(self.max_tweets), "tweets.")
            return False
            
    #method for on_error()
    def on_error(self, status):
        print("ERROR")
        if(status==420):
            print("Error ", status, "rate limited")
            return False


In Listener...


### Tweets mining

In [12]:
hashname=input("Enter the hash name, such as #womensrights: ") 
numtweets=eval(input("How many tweets do you want to get?: "))

if(hashname[0]=="#"):
    nohashname=hashname[1:] #remove the hash
else:
    nohashname=hashname
    hashname="#"+hashname

#Create a file for any hash name    
hfilename="file_"+nohashname+".txt"
# raw file
rawfile="file_rawtweets_"+nohashname+".txt"

twitter_stream = Stream(auth, Listener(numtweets, hfilename, rawfile))

#twitter_stream.filter(track=['#womensrights'])
twitter_stream.filter(track=[hashname])
print("Twitter files created....")

Enter the hash name, such as #womensrights: covid19
How many tweets do you want to get?: 50
50
In on_data 1
In on_data in try
RT @LauraSmithCrewe: Politician’s are going to now get distracted with Brexit wars whilst furlough ends, redundancies rise, covid rates soa… 

In on_data 2
In on_data in try
In sha Allah ❤❤❤ 

In on_data 3
In on_data in try
RT @ImranKhanPTI: کل سکول واپسی پر ہم لاکھوں بچوں کو خوش آمدید کہیں گے۔ یہ یقینی بنانا کہ ہر بچہ حصولِ علم کیلئےبحفاظت سکول جاسکے، ہماری تر… 

In on_data 4
In on_data in try
RT @Resist2Exist313: Large #antilockdownprotest took place in Bulgaria 🇧🇬 where protestors called on their government to resign accusing th… 

In on_data 5
In on_data in try
RT @RedLopezDiaz: Se confirma la reinfección por #COVID19 de una trabajadora del Sistema Estatal de Salud en #Puebla, informó el titular de… 

In on_data 6
In on_data in try
RT @marklouiegi55: CHED DOES NOT CARE ABOUT THE MENTAL AND PHYSICAL HEALTH OF STUDENTS PERIODT 

In on_data 7
In on_data in try


### Data Preprocessing

In [ ]:
! ls

In [14]:
linecount=0
hashcount=0
wordcount=0
BagOfWords=[]
BagOfHashes=[]
BagOfLinks=[]

tweetsfile=hfilename
#tweetsfile = 'file_covid19.txt'

filename="CleanText.csv"
NEWFILE=open(filename,"w")

## In the first row, create a column called Label and a column Text...
ToWrite="Label,Text\n"

## Write this to new empty cs v file
NEWFILE.write(ToWrite)

## Close it up
NEWFILE.close()
NEWFILE=open(filename, "a")



with open(tweetsfile, 'r') as file:
    for line in file:
        print(line,"\n")
        tweetSplitter = TweetTokenizer(strip_handles=True, reduce_len=True)
        WordList=tweetSplitter.tokenize(line)

        regex1=re.compile('^#.+')
        regex2=re.compile('[^\W\d]') #no numbers
        regex3=re.compile('^http*')
        regex4=re.compile('.+\..+')
        NewList =[]

        for item in WordList:
            if(len(item)>2):
                if((re.match(regex1,item))):
                    #print(item)
                    newitem=item[1:] #remove the hash
                    BagOfHashes.append(newitem)
                    hashcount=hashcount+1
                     # write each cleaned tweet on a line.    
                    NewList.append(newitem)

                elif(re.match(regex2,item)):
                    if(re.match(regex3,item) or re.match(regex4,item)):
                        BagOfLinks.append(item)
                    else:
                        BagOfWords.append(item)
                        wordcount=wordcount+1
                        # write each cleaned tweet on a line.
                        NewList.append(item)
                else:
                    pass
            else:
                pass
           
        
         # write each cleaned tweet on a line. 
        Text=" ".join(NewList)
        Text="\t"+","+Text + "\n"
        NEWFILE.write(Text)
        
    
NEWFILE.close()

In [16]:
import pandas as pd
df = pd.read_csv(filename)
df.head()

,Label,Text
0,\t,Politician are going now get distracted with B...
1,\t,You and that poor innocent non-druggie CHED DO...
2,\t,OBOR
3,\t,NaN
4,\t,COVID19 önlenmesi kontrolünün düzenli bir uygu...
